In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt



In [ ]:
df = pd.read_csv("dataset.csv")


In [ ]:
# features(X) / target(y) 분리
X = df.iloc[:, :-1]
y = df.iloc[:, -1]


In [ ]:
#  training/test 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
#  CatBoost ensemble uncertainty 함수 정의
def catboost_ensemble_uncertainty(
    X_train, y_train, X_test,
    seeds=[0,1,2,3,4],
    init_params=None,
    fit_params=None
):
    init_params = init_params or {}
    fit_params  = fit_params  or {}

    all_preds = []
    for seed in seeds:
        model = CatBoostRegressor(
            random_seed=seed,
            **init_params
        )
        model.fit(
            X_train, y_train,
            **fit_params
        )
        preds = model.predict(X_test)
        all_preds.append(preds)

    all_preds = np.stack(all_preds, axis=0)
    mean_preds = all_preds.mean(axis=0)
    std_preds  = all_preds.std(axis=0)
    return mean_preds, std_preds


In [ ]:
# hyperparameter 및 seed 설정
init_params = {
    "iterations": 500,
    "learning_rate": 0.1,
    "depth": 6
}
fit_params = {
    "verbose": False
}
seeds = [0, 1, 2, 3, 4]  # 앙상블 크기 = 5


In [ ]:

#  호출 및 결과 출력
mean_cb, std_cb = catboost_ensemble_uncertainty(
    X_train, y_train, X_test,
    seeds=seeds,
    init_params=init_params,
    fit_params=fit_params
)


In [ ]:

# 예측값과 불확실성 확인 (첫 10개 샘플)
for i in range(10):
    print(f"샘플 {i:2d}: 예측={mean_cb[i]:.3f}, 불확실성(std)={std_cb[i]:.3f}")


In [ ]:
# 요약 통계 출력 (모델 전체의 불확실성 평가)
print("테스트 샘플 수       :", len(std_cb))
print("평균 불확실성        :", std_cb.mean():.3f)
print("중앙값 불확실성       :", np.median(std_cb):.3f)
print("최소 불확실성        :", std_cb.min():.3f)
print("최대 불확실성        :", std_cb.max():.3f)

In [ ]:
# 히스토그램으로 분포 시각화하기
plt.figure(figsize=(8, 5))
plt.hist(std_cb, bins=30, edgecolor='k')
plt.xlabel("불확실성 (std)")
plt.ylabel("샘플 수")
plt.title("테스트 세트 불확실성 분포")
plt.show()